In [ ]:
# %% Calculus 2 - Section 4.26
#    Code challenge: Lebesgue in Blefuscu - I

# This code pertains to a calculus course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/pycalc2_x
# The code in this repository is developed to solve the exercises provided along
# the course, and it has been written partially indepentently and partially
# from the code developed by the course instructor.


In [2]:
import numpy             as np
import sympy             as sym
import matplotlib.pyplot as plt
import math

from scipy.signal                     import find_peaks
from IPython.display                  import display,Math
from google.colab                     import files
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Exercise 1
#    Compute the Riemann approximation of the function shown in the video

# Function
def f(x):
    return x**2 + np.cos(2*np.pi*x)/5

# Bounds, dx and areaa approximation
a = 0
b = 1
n = 12
delta_x = (b-a)/n

delta_border  = np.array([a+delta_x*i for i in range(n+1)])
area_midpoint = np.sum(f((delta_border[:-1]+delta_border[1:])/2)) * delta_x
xx = np.linspace(a-0.1,b+0.1,501)

# Plotting
phi = (1 + np.sqrt(5)) / 2
plt.figure(figsize=(5*phi,5))

plt.plot(xx,f(xx),'k',linewidth=2)
plt.axvline(x=a,color='r',linestyle=':',linewidth=0.8,label=r'$a$ = %.2f'%a)
plt.axvline(x=b,color='r',linestyle=':',linewidth=0.8,label=r'$b$ = %.2f'%b)
plt.axhline(y=0,color='grey',linestyle=':',linewidth=0.8)
plt.gca().set(xlabel='x',ylabel='y=f(x)',xlim=xx[[0,-1]],ylim=[-0.1,1.5])

cmap = plt.cm.plasma(np.linspace(.1,.9,n))

for i in range(n):

  # Midpoint rule
  b = delta_border[i] + delta_x/2
  plt.fill_between([b-delta_x/2,b+delta_x/2],[f(b),f(b)],color=cmap[i],alpha=.66)

# plot titles
plt.title(f'Midpoint rule: net area = {area_midpoint:.3f}, $\\Delta$x = {delta_x:.3f}')

plt.tight_layout()

plt.savefig('fig9_codechallenge_26_exercise_1.png')
plt.show()
files.download('fig9_codechallenge_26_exercise_1.png')


In [ ]:
# %% Exercise 2
#    Compute the Lebesgue approximation of the function from exercise 1

# Partition x axis
xx         = np.linspace(a-0.1,b+0.1,501)
domain_n   = 1000
domain_pts = np.linspace(a,b,domain_n)
delta_x    = (b-a) / domain_n

# Evaluate function at those points
y       = f(domain_pts)
min_val = np.min(y)
max_val = np.max(y)

# Boundaries
range_pts = np.linspace(min_val,max_val,n+1)
delta_y   = range_pts[1] - range_pts[0]

# Preallocate Lebesgue approximation
lebesgue = 0

# Plotting
phi = (1 + np.sqrt(5)) / 2
plt.figure(figsize=(5*phi,5))

plt.plot(xx,f(xx),'k',linewidth=2)
cmapA  = plt.cm.plasma(np.linspace(.1,.9,n))
darken = lambda rgb, factor=0.7: [c * factor for c in rgb]
cmapP  = darken(cmapA)

for i in range(n):

    # Find points where the function is within the current partition
    in_partition = (y >= range_pts[i]) & (y < range_pts[i+1])

    # Measure the "size" of that set
    measure = np.sum(in_partition) * delta_x

    # The average function value on this set
    average_value = (range_pts[i] + range_pts[i+1]) / 2

    # Sum this set to the integral approximation
    lebesgue += average_value * measure

    # Find the contiguous groups in in_partition
    in_partition_diff = np.diff(in_partition.astype(int))
    group_starts      = np.where(in_partition_diff == 1)[0] + 1   # start points of groups
    group_ends        = np.where(in_partition_diff == -1)[0]      # end points of groups

    # In case a group starts/ends at the integration bounds
    if in_partition[0]:
        group_starts = np.insert(group_starts,0,0)
    if in_partition[-1]:
        group_ends = np.append(group_ends,len(in_partition) - 1)


    # Loop over groups and draw rectangles
    for start, end in zip(group_starts,group_ends):

        # Visualization option "a"
        x1,x2 = domain_pts[start],domain_pts[end]
        y1,y2 = range_pts[i],range_pts[i+1]

        # Visualization option "b"
        x1,x2 = domain_pts[start],domain_pts[end]
        y1,y2 = 0,range_pts[i+1]

        # Visualization option "c"
        x1    = a if f(domain_pts[end])<f(domain_pts[start]) else domain_pts[start]
        x2    = b if f(domain_pts[end])>f(domain_pts[start]) else domain_pts[end]
        y1,y2 = range_pts[i],range_pts[i+1]

        # Draw the patch
        plt.fill_between([x1,x2],y1,y2,edgecolor=cmapP[i],facecolor=cmapA[i],alpha=0.75)

plt.gca().set(xlabel='x',ylabel=r'$y = x^2+\cos(2\pi x)/5$')
plt.title(r'Net area = %.3f $\Delta$y=%.3f' %(lebesgue,delta_y))

plt.axhline(y=0,color='gray',linestyle=':',linewidth=0.8)
plt.axvline(x=a,color='gray',linestyle=':',linewidth=0.8)
plt.axvline(x=b,color='gray',linestyle=':',linewidth=0.8)
plt.xlim(xx[[0,-1]])
plt.yticks(range_pts)
plt.tight_layout()

plt.savefig('fig10_codechallenge_26_exercise_2.png')
plt.show()
files.download('fig10_codechallenge_26_exercise_2.png')
